## Logistic Regression via Newton's Method

* Newton's Method Functions:  
     - **logfunc(x)**: finds the logistic function  $$f(x) = \frac{1}{1+e^{-\beta^Tx}}$$
     - **newton_method_lrstep(beta0,y,X)**: Updates Newton's Method, $$\beta^{(t+1)} = \beta^{(t)} + H^{-1}\nabla{l(\beta)}$$  where $l(\beta)$ is the log-likelihood of $\beta$ for a Logistic Regression  
     - **tolerance_check(beta0,beta,eps)**: Checks if it converges to a set threshold, $$|\beta^{(t)}-\beta^{t+1}|<\epsilon$$
     - **newton_method_logreg(beta0,y,X,eps)**: Newton's Method for Logistic Regression

In [2]:
import numpy as np
import scipy.linalg as linalg 

def logfun(x): # logistic function
    p = 1/(1+np.exp(-x)) # probability
    return p

def newton_method_lrstep(beta0,y,X, method):
    betaX = np.dot(X,beta0) # beta*X
    yx = logfun(betaX) # y_hat
    y_hat = np.array(yx,ndmin=2) # converts to y_hat array
    gradient = np.dot(X.T, (y-y_hat)) # gradient

    ny_hat = 1-y_hat
    d = np.dot(y_hat,ny_hat.T)
    diag = np.diag(d)
    d = np.diag(diag)
    hessian = X.to_numpy().T.dot(d).dot(X.to_numpy()) # hessian matrix
    if method == "LU":
        hessian_inv = np.linalg.inv(hessian)
    elif method == "QR":
        n = hessian.shape[1]
        Q,R = np.linalg.qr(hessian)
        Rinv = linalg.solve_triangular(R,np.identity(n))
        hessian_inv = np.dot(Rinv,Q.T)
    else:
        hessian_inv = np.linalg.inv(hessian) ## REPLACE FOR CHOLEVSKY DECOMPOSITION ##

    gd = np.dot(hessian_inv,gradient) # finds the step direction

    beta = beta0 + gd # updates coefficients

    return beta

def tolerance_check(beta0, beta, eps):
    diff = np.abs(beta0-beta) # norm 
    if np.any(diff>eps): # if norm crosses threshold
        return False
    else:
        return True
    
def newton_method_logreg(beta0, y, X,eps,method):
    iterations = 0 # initial iterations
    converge = False # sets converge to false
    while not converge: # while converge is false
        beta = newton_method_lrstep(beta0,y,X,method) # finds new beta
        converge = tolerance_check(beta0,beta,eps) # checks convergence
        beta0 = beta # updates beta
        iterations +=1 # updates iterations
        print ("Iteration: {}".format(iterations))
    return beta



**Split Dataset**  

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

dropout = pd.read_csv("../Datasets/schooldropout.csv", sep=";")
dropout['Target'].replace(['Dropout', 'Graduate',"Enrolled"],[0, 1,1], inplace=True)

X = dropout.drop(['Target'],axis=1)
y = dropout[["Target"]]

X_train,x_test,y_train, y_test = train_test_split(X,y ,random_state=100, test_size=0.20, shuffle=True)

**Initial Test**

In [4]:
import numpy as np

beta0 = np.zeros((36,1))
eps = 10**(-3)
beta = newton_method_logreg(beta0,y_train,X_train,eps,method="LU")
beta

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6


array([[ 2.00058713e-01],
       [-1.44829319e-03],
       [-8.47128733e-02],
       [-1.01993413e-04],
       [ 5.03734035e-02],
       [ 1.08353844e-02],
       [-4.21872230e-03],
       [-3.82197232e-02],
       [-1.43403009e-02],
       [ 9.19269825e-03],
       [ 1.51162903e-02],
       [-5.78843199e-03],
       [ 5.43759909e-03],
       [-2.74915159e-01],
       [-5.27119976e-01],
       [-4.45410756e-01],
       [ 2.30739391e+00],
       [-2.11933927e-01],
       [ 5.93439044e-01],
       [-4.43493981e-02],
       [ 2.25226491e+00],
       [-1.07333893e-01],
       [-2.08974485e-02],
       [ 1.92732815e-02],
       [ 2.81507878e-01],
       [-6.37193801e-02],
       [ 1.21422582e-01],
       [-2.30988212e-01],
       [-4.61497530e-01],
       [ 3.50939614e-02],
       [ 6.06727214e-01],
       [ 7.87625927e-02],
       [ 1.06016918e-01],
       [-5.48824012e-02],
       [-1.19697421e-02],
       [ 2.05755568e-02]])

**Logistic Regression Comparison**

In [5]:
import statsmodels.api as sm
logit_model=sm.Logit(y_train,X_train)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.315118
         Iterations 7
                                        Results: Logit
Model:                        Logit                      Pseudo R-squared:           0.501    
Dependent Variable:           Target                     AIC:                        2302.4074
Date:                         2023-03-15 18:33           BIC:                        2524.5850
No. Observations:             3539                       Log-Likelihood:             -1115.2  
Df Model:                     35                         LL-Null:                    -2236.6  
Df Residuals:                 3503                       LLR p-value:                0.0000   
Converged:                    1.0000                     Scale:                      1.0000   
No. Iterations:               7.0000                                                          
-----------------------------------------------------------------------------------

**Testing**  
Taken from online resource

In [6]:
def predict_lr(X,y,beta):
    xbeta = np.dot(X,beta)
    predict = np.array(logfun(xbeta))
    threshold = 0.5*np.ones((predict.shape[0],1))
    pred_class = np.greater(predict,threshold)
    accuracy = np.count_nonzero(np.equal(pred_class, y))/pred_class.shape[0]
    return accuracy

In [7]:
predict_lr(x_test,y_test,beta)

0.8779661016949153